In [19]:
import numpy as np

# Load your data
train_images = np.load('/kaggle/input/sudokotask1/train/imgs-001.npy')  # Shape: (50000, 252, 252)
train_labels = np.load('/kaggle/input/sudokotask1/train/known_values_labels.npy')  # Shape: (50000, 41, 3)
test_images = np.load('/kaggle/input/sudokotask1/test/imgs.npy')    # Shape: (10000, 252, 252)

In [ ]:
import cv2

import numpy as np

# def preprocess_images_batch(images, batch_size):
#     num_images = images.shape[0]
#     cell_size = 28
#     for batch_start in range(0, num_images, batch_size):
#         batch_end = min(batch_start + batch_size, num_images)
#         processed_batch = np.zeros(((batch_end - batch_start) * 81, cell_size, cell_size))

#         for idx in range(batch_start, batch_end):
#             for row in range(9):
#                 for col in range(9):
#                     cell = images[idx, row * cell_size:(row + 1) * cell_size, col * cell_size:(col + 1) * cell_size]
#                     processed_cell = cell / 255.0
#                     processed_batch[(idx - batch_start) * 81 + row * 9 + col] = processed_cell

#         # Here you can save each batch to disk or yield it if using a generator
#         yield processed_batch

# # Example usage
# batch_size = 100  # Adjust this based on your system's memory capacity
# for processed_batch in preprocess_images_batch(train_images, batch_size):
#     # Save or process each batch
#     pass

# for processed_batch in preprocess_images_batch(train_images, batch_size):
#     # Save or process each batch
#     pass
# def preprocess_images_simple(images):
#     num_images = images.shape[0]
#     cell_size = 28
#     processed_images = np.zeros((num_images * 81, cell_size, cell_size))

#     for idx in range(num_images):
#         for row in range(9):
#             for col in range(9):
#                 # Extracting each cell
#                 cell = images[idx, row * cell_size:(row + 1) * cell_size, col * cell_size:(col + 1) * cell_size]
#                 # Normalize the cell
#                 processed_cell = cell / 255.0
#                 # Store in processed_images
#                 processed_images[idx * 81 + row * 9 + col] = processed_cell

#     return processed_images
def preprocess_images_simple(images, new_size=(14, 14)):
    num_images = images.shape[0]
    processed_images = np.zeros((num_images * 81, new_size[0], new_size[1]))

    for idx in range(num_images):
        for row in range(9):
            for col in range(9):
                # Extract each cell
                cell = images[idx, row*28:(row+1)*28, col*28:(col+1)*28]
                # Resize the cell
                resized_cell = cv2.resize(cell, new_size, interpolation=cv2.INTER_AREA)
                # Normalize the cell
                processed_cell = resized_cell / 255.0
                # Store in processed_images
                processed_images[idx * 81 + row * 9 + col] = processed_cell

    return processed_images

# Process the entire datasets with reduced resolution
train_images_processed = preprocess_images_simple(train_images)
test_images_processed = preprocess_images_simple(test_images)

# # Process the entire datasets
# train_images_processed = preprocess_images_simple(train_images)
# test_images_processed = preprocess_images_simple(test_images)


# train_images_processed = preprocess_images_batch(train_images, batch_size)
# test_images_processed = preprocess_images_batch(test_images, batch_size)
# def preprocess_images(images):
#     num_images = images.shape[0]
#     # Each image is divided into 9x9 cells, each cell is approximately 28x28
#     cell_size = 28
#     processed_images = np.zeros((num_images * 81, cell_size, cell_size))

#     for idx in range(num_images):
#         for row in range(9):
#             for col in range(9):
#                 # Extracting each cell
#                 cell = images[idx, row * cell_size:(row + 1) * cell_size, col * cell_size:(col + 1) * cell_size]
#                 # Normalize the cell
#                 processed_cell = cell / 255.0
#                 # Store in processed_images
#                 processed_images[idx * 81 + row * 9 + col] = processed_cell

#     return processed_images

# train_images_processed = preprocess_images(train_images)
# test_images_processed = preprocess_images(test_images)

In [ ]:
# We need to convert train_labels to a format suitable for training
def prepare_labels(labels):
    # Flatten the labels into a 1D array
    # Each image contributes 81 (9x9) cells
    num_images = len(labels)
    new_labels = np.zeros((num_images * 81), dtype=int)

    for i, label in enumerate(labels):
        for cell in label:
            x, y, value = cell
            x, y, value = int(x), int(y), int(value)  # Ensure integers
            if 0 <= x < 9 and 0 <= y < 9:  # Check for bounds
                # Convert 2D coordinates to 1D index and set the value
                new_labels[i * 81 + x * 9 + y] = value

    return new_labels

train_labels_processed = prepare_labels(train_labels)



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# def build_model():
#     model = Sequential([
#         Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'),
#         MaxPooling2D(2, 2),

#         Conv2D(64, (3, 3), activation='relu', padding='same'),
#         MaxPooling2D(2, 2),

#         Conv2D(128, (3, 3), activation='relu', padding='same'),
#         MaxPooling2D(2, 2),

#         Flatten(),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(10, activation='softmax')
#     ])
#     return model
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(14, 14, 1), padding='same'),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(64, activation='relu'),  # Reduced number of neurons
        Dropout(0.5),
        Dense(10, activation='softmax')  # 10 classes for digits 0-9
    ])
    return model

# model = build_model()
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model.summary()

model = build_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming train_images_processed and train_labels_processed are your full datasets
# Split the first 25,000 for training and the rest for potential validation and testing
train_images_subset = train_images_processed[:50000]
train_labels_subset = train_labels_processed[:50000]

# Further split the training data to create a validation set
# Here, I'll use 20% of the 25,000 images for validation, but you can adjust this percentage
train_images_final, val_images, train_labels_final, val_labels = train_test_split(
    train_images_subset, train_labels_subset, test_size=0.2, random_state=42)

# Reshape the datasets for the CNN
train_images_final_reshaped = np.expand_dims(train_images_final, axis=-1)
val_images_reshaped = np.expand_dims(val_images, axis=-1)


In [ ]:
# Train the model
model.fit(train_images_final_reshaped, train_labels_final, 
          validation_data=(val_images_reshaped, val_labels),
          epochs=10, batch_size=8)


In [ ]:
# Reshape the test_images for prediction
test_images_reshaped = np.expand_dims(test_images_processed, axis=-1)

# Predict
predictions = model.predict(test_images_reshaped)


In [ ]:
def create_submission_file(predictions):
    with open('submission2.csv', 'w') as f:
        f.write('id,value\n')
        for img_id in range(len(predictions)):
            for y in range(9):
                for x in range(9):
                    cell_id = f"{img_id}_{x}{y}"
                    value = int(predictions[img_id][x * 9 + y])
                    f.write(f"{cell_id},{value}\n")

create_submission_file(predictions)

In [ ]:
def create_submission_file(predictions, num_images):
    with open('submission.csv', 'w') as f:
        f.write('id,value\n')
        for img_id in range(num_images):
            for position in range(81):  # Iterate over each cell in the grid
                x = position // 9  # x-coordinate (row)
                y = position % 9   # y-coordinate (column)
                cell_id = f"{img_id}_{x}{y}"
                # Calculate the index in the predictions array
                pred_index = img_id * 81 + position
                # Get the most likely digit for this cell
                value = np.argmax(predictions[pred_index])
                f.write(f"{cell_id},{value}\n")

create_submission_file(predictions, num_images=10000)  # Replace 50000 with your actual number of images


In [ ]:
print(predictions.shape)